<a href="https://colab.research.google.com/github/svgoudar/data_science_tutorial/blob/main/ML/8.SVM/Practicals/4.Support_Vector_Regression_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Support Vector Regression Implementation

In [54]:
## Dataset (Tips Dataset)
import seaborn as sns
df=sns.load_dataset('tips')

In [55]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [57]:
df['sex'].value_counts()

,count
sex,
Male,157
Female,87


In [58]:
df['day'].value_counts()

,count
day,
Sat,87
Sun,76
Thur,62
Fri,19


In [59]:
df['smoker'].value_counts()

,count
smoker,
No,151
Yes,93


In [60]:
df['time'].value_counts()

,count
time,
Dinner,176
Lunch,68


In [61]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [62]:
## independnent and dependent features
X=df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y=df['total_bill']

In [63]:
##train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=10)

In [64]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,Male,Yes,Sat,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
68,2.01,Male,No,Sat,Dinner,2
184,3.00,Male,Yes,Sun,Dinner,2


## Feature Encoding(Label Encoding And Onehot Encoding)

In [65]:
from sklearn.preprocessing import LabelEncoder

In [66]:
le1=LabelEncoder()
le2=LabelEncoder()
le3=LabelEncoder()

In [67]:
import warnings
warnings.filterwarnings('ignore')
X_train['sex']=le1.fit_transform(X_train['sex'])
X_train['smoker']=le2.fit_transform(X_train['smoker'])
X_train['time']=le3.fit_transform(X_train['time'])

In [68]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2


In [69]:
X_test['sex']=le1.transform(X_test['sex'])
X_test['smoker']=le2.transform(X_test['smoker'])
X_test['time']=le3.transform(X_test['time'])

In [70]:
X_test.head()

,tip,sex,smoker,day,time,size
162,2.00,0,0,Sun,0,3
60,3.21,1,1,Sat,0,2
61,2.00,1,1,Sat,0,2
63,3.76,1,1,Sat,0,4
69,2.09,1,1,Sat,0,2


In [71]:
## Onehot encoding--- ColumnTrnasformer

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [72]:
ct=ColumnTransformer(transformers=[('onehot',OneHotEncoder(drop='first'),[3])],
                             remainder='passthrough')

**Explanation:**

This code is setting up a `ColumnTransformer` to apply one-hot encoding to a specific column in your dataset while handling the other columns differently. Let's dissect each part:

1.  **`from sklearn.compose import ColumnTransformer`**
    *   This line imports the `ColumnTransformer` class from the `sklearn.compose` module. `ColumnTransformer` is a tool that allows you to apply different transformations to different columns of your dataset. This is very useful in situations where you want to treat numerical and categorical columns differently.

2.  **`from sklearn.preprocessing import OneHotEncoder`**
    *   This imports the `OneHotEncoder` class, which is used for transforming categorical data into a numerical format that machine learning algorithms can understand.

3.  **`ct = ColumnTransformer(...)`**
    *   Here, we're creating an instance of the `ColumnTransformer` and configuring how it should operate.
    *   **`transformers=[(...)]`**: This argument specifies a list of transformations to apply. Each transformation is defined as a tuple:
        *   **`('onehot', OneHotEncoder(drop='first'), [3])`**: This tuple defines a single transformation:
            *   **`'onehot'`**: This is the name we're giving to this transformation. It's simply an identifier that you can choose.
            *   **`OneHotEncoder(drop='first')`**: This is the actual transformation. We're creating an instance of `OneHotEncoder` with a specific configuration:
                *   **`drop='first'`**: This is a crucial parameter. It tells the `OneHotEncoder` to drop the first category from each feature it encodes. By dropping the first category it removes one degree of freedom. This can improve your model's performance. (See result \[1] and \[3]).
            *   **`[3]`**: This is a list containing the column index to which this transformation should be applied. In this case, it's `3`, which means the fourth column (remember, indexing starts at `0`). Based on the previous code in the notebook column `3` is the column `day`.
    * **`remainder='passthrough'`**:
      * This argument specifies what to do with the columns that are not specified in the `transformers` list.
        * **`'passthrough'`**: indicates that those columns should be passed through without any transformation. This means that all columns that are not `day` will remain unchanged in the dataset after this transformation.

**In essence, this code is doing the following:**

1.  It identifies the fourth column (index 3) of your data, which is column `day`.
2.  It takes this column, and for any unique value that column has it will create a new column. This will happen only with the column day and the rest of the columns will not be affected.
3.  It applies one-hot encoding to this column (`day`), creating new binary columns for each unique value.
4.  It drops the first encoded category to avoid multi-collinearity.
5.  It passes all other columns through the transformer without modification.

**Why is this useful?**

*   **Encoding Categorical Data:** Machine learning algorithms generally work best with numerical data. `OneHotEncoder` converts categorical features (like 'day') into a numerical representation.
*   **Avoiding Multi-collinearity:** `drop='first'` helps prevent multi-collinearity, a situation where features are highly correlated, which can negatively affect the performance of some machine learning models.
*   **Targeting Specific Columns:** `ColumnTransformer` allows you to be selective about which transformations you apply to which columns, giving you fine-grained control over your data preprocessing.

I hope this explanation is helpful! Let me know if you have any other questions.

In [73]:
import sys
import numpy
import pandas as pd
numpy.set_printoptions(threshold=sys.maxsize)
X_train=ct.fit_transform(X_train)

In [74]:
X_train

array([[ 1.  ,  0.  ,  0.  ,  1.76,  1.  ,  1.  ,  0.  ,  2.  ],
       [ 0.  ,  1.  ,  0.  ,  1.66,  1.  ,  0.  ,  0.  ,  3.  ],
       [ 0.  ,  1.  ,  0.  ,  3.5 ,  1.  ,  0.  ,  0.  ,  3.  ],
       [ 1.  ,  0.  ,  0.  ,  2.01,  1.  ,  0.  ,  0.  ,  2.  ],
       [ 0.  ,  1.  ,  0.  ,  3.  ,  1.  ,  1.  ,  0.  ,  2.  ],
       [ 0.  ,  1.  ,  0.  ,  4.  ,  0.  ,  1.  ,  0.  ,  2.  ],
       [ 0.  ,  1.  ,  0.  ,  5.2 ,  0.  ,  0.  ,  0.  ,  4.  ],
       [ 0.  ,  0.  ,  1.  ,  4.  ,  1.  ,  1.  ,  1.  ,  4.  ],
       [ 0.  ,  0.  ,  1.  ,  5.  ,  1.  ,  0.  ,  1.  ,  5.  ],
       [ 1.  ,  0.  ,  0.  ,  3.  ,  0.  ,  0.  ,  0.  ,  2.  ],
       [ 1.  ,  0.  ,  0.  ,  1.75,  1.  ,  0.  ,  0.  ,  2.  ],
       [ 0.  ,  1.  ,  0.  ,  3.12,  1.  ,  0.  ,  0.  ,  4.  ],
       [ 0.  ,  0.  ,  1.  ,  3.  ,  0.  ,  0.  ,  0.  ,  2.  ],
       [ 1.  ,  0.  ,  0.  ,  5.  ,  1.  ,  0.  ,  0.  ,  3.  ],
       [ 0.  ,  0.  ,  1.  ,  4.  ,  1.  ,  0.  ,  1.  ,  2.  ],
       [ 1.  ,  0.  ,  0.

In [75]:
X_test=ct.transform(X_test)

In [76]:
X_test

array([[0.  , 1.  , 0.  , 2.  , 0.  , 0.  , 0.  , 3.  ],
       [1.  , 0.  , 0.  , 3.21, 1.  , 1.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 2.  , 1.  , 1.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 3.76, 1.  , 1.  , 0.  , 4.  ],
       [1.  , 0.  , 0.  , 2.09, 1.  , 1.  , 0.  , 2.  ],
       [0.  , 0.  , 1.  , 5.  , 1.  , 1.  , 1.  , 2.  ],
       [0.  , 1.  , 0.  , 3.51, 1.  , 0.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 5.16, 1.  , 1.  , 0.  , 4.  ],
       [0.  , 1.  , 0.  , 5.  , 1.  , 0.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 3.6 , 1.  , 0.  , 0.  , 3.  ],
       [0.  , 1.  , 0.  , 5.65, 1.  , 1.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 2.5 , 0.  , 1.  , 0.  , 3.  ],
       [0.  , 0.  , 1.  , 1.44, 1.  , 0.  , 1.  , 2.  ],
       [1.  , 0.  , 0.  , 3.09, 0.  , 1.  , 0.  , 4.  ],
       [0.  , 1.  , 0.  , 2.  , 1.  , 0.  , 0.  , 4.  ],
       [0.  , 0.  , 1.  , 1.36, 0.  , 0.  , 1.  , 3.  ],
       [0.  , 0.  , 1.  , 2.  , 0.  , 0.  , 1.  , 2.  ],
       [0.  , 0.  , 1.  , 1.68,

In [77]:
pd.DataFrame(X_test).head()

,0,1,2,3,4,5,6,7
0,0.0,1.0,0.0,2.00,0.0,0.0,0.0,3.0
1,1.0,0.0,0.0,3.21,1.0,1.0,0.0,2.0
2,1.0,0.0,0.0,2.00,1.0,1.0,0.0,2.0
3,1.0,0.0,0.0,3.76,1.0,1.0,0.0,4.0
4,1.0,0.0,0.0,2.09,1.0,1.0,0.0,2.0


In [78]:
## SVR--Support Vector Regression
from sklearn.svm import SVR
svr=SVR()

In [80]:
svr.fit(X_train,y_train)

SVR()

In [81]:
y_pred=svr.predict(X_test)

In [82]:
from sklearn.metrics import r2_score,mean_absolute_error
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.46028114561159283
4.1486423210190235


## Hyperparameter Tuning using GridSearch CV

In [83]:
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

In [84]:
grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.067 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.058 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.145 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.025 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.089 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.013 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.021 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.010 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.124 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.050 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.053 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [85]:
grid.best_params_

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

In [86]:
grid_prediction=grid.predict(X_test)

In [87]:
from sklearn.metrics import r2_score,mean_absolute_error
print(r2_score(y_test,grid_prediction))
print(mean_absolute_error(y_test,grid_prediction))

0.5081618245078687
3.8685147526100234
